In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas._libs.tslibs import dtypes, timestamps

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
df = pd.read_csv('./control_subset_simple/resnet50_2_64_1.csv', skiprows=[0,1,2,3,4])
df_gpu = pd.read_csv('./control_subset_simple/resnet50_2_64_1_gpu.csv', dtype={"utilization.gpu": float})

# df_gpu.drop(columns=['name', 'pci.bus_id'], inplace=True)
# rename columns
df.rename(columns={
'read':'read_io_total',
'writ':'write_io_total',
'time':'system_time',
'usr':'usr_cpu_usage',
'sys':'sys_cpu_usage',
'idl':'idl_cpu_usage',
'wai':'wai_cpu_usage',
'stl':'stl_cpu_usage',
'read.1':'read_dsk_total',
'writ.1':'writ_dsk_total',
'used':'used_memory',
'free':'free_memory',
'buff':'buff_memory',
'cach':'cach_memory',
'recv':'recv_net_total',
'send':'send_net_total',
'in':'in_paging',
'out':'out_paging'}, inplace=True)

df_gpu.rename(columns={
' temperature.gpu':'temperature_gpu',
' utilization.gpu [%]':'utilization_gpu',
' utilization.memory [%]':'utilization_memory',
' memory.total [MiB]':'memory_total',
' memory.free [MiB]':'memory_free',
' memory.used [MiB]':'memory_used'}, inplace=True)

In [ ]:
print(df.head())
print(df_gpu.head())

In [ ]:
# Totals

print(f"Read IO Total: {df['read_io_total'].sum()}")
print(f"Write IO Total: {df['write_io_total'].sum()}")
print(f"Read Disk Total: {df['read_dsk_total'].sum()}")
print(f"Write Disk Total: {df['writ_dsk_total'].sum()}")
print(f"Used Memory: {df['used_memory'].sum()}")
print(f"Free Memory: {df['free_memory'].sum()}")
print(f"Buffer Memory: {df['buff_memory'].sum()}")
print(f"Cache Memory: {df['cach_memory'].sum()}")
print(f"Receive Net Total: {df['recv_net_total'].sum()}")
print(f"Send Net Total: {df['send_net_total'].sum()}")
print()
print (f"Total GPU Utilization Memory (%): {df_gpu['utilization_memory'].sum()}")
print (f"Total GPU Memory Total (MiB): {df_gpu['memory_total'].sum()}")
print (f"Total GPU Memory Free (MiB): {df_gpu['memory_free'].sum()}")
print (f"Total GPU Memory Used (MiB): {df_gpu['memory_used'].sum()}")

In [ ]:
# Means

print(f"Read IO Average: {df['read_io_total'].mean()}")
print(f"Write IO Average: {df['write_io_total'].mean()}")
print(f"Read Disk Average: {df['read_dsk_total'].mean()}")
print(f"Write Disk Average: {df['writ_dsk_total'].mean()}")
print(f"Used Memory Average: {df['used_memory'].mean()}")
print(f"Free Memory Average: {df['free_memory'].mean()}")
print(f"Buffer Memory Average: {df['buff_memory'].mean()}")
print(f"Cache Memory Average: {df['cach_memory'].mean()}")
print(f"Receive Net Average: {df['recv_net_total'].mean()}")
print(f"Send Net Average: {df['send_net_total'].mean()}")
print(f"User CPU Usage Average: {df['usr_cpu_usage'].mean()}")
print(f"System CPU Usage Average: {df['sys_cpu_usage'].mean()}")
print(f"Idle CPU Usage Average: {df['idl_cpu_usage'].mean()}")
print(f"Wait CPU Usage Average: {df['wai_cpu_usage'].mean()}")
print()
print (f"GPU Temperature Average: {df_gpu['temperature_gpu'].mean()}")
print (f"GPU Utilization Average (%): {df_gpu['utilization_gpu'].mean()}")
print (f"GPU Utilization Memory Average (%): {df_gpu['utilization_memory'].mean()}")
print (f"GPU Memory Total Average (MiB): {df_gpu['memory_total'].mean()}")
print (f"GPU Memory Free Average (MiB): {df_gpu['memory_free'].mean()}")
print (f"GPU Memory Used Average (MiB): {df_gpu['memory_used'].mean()}")

In [ ]:
# Max and Min values
print(f"Max Read IO: {df['read_io_total'].max()} ; Min Read IO: {df['read_io_total'].min()}")
print(f"Max Write IO: {df['write_io_total'].max()} ; Min Write IO: {df['write_io_total'].min()}")
print(f"Max Read Disk: {df['read_dsk_total'].max()} ; Min Read Disk: {df['read_dsk_total'].min()}")
print(f"Max Write Disk: {df['writ_dsk_total'].max()} ; Min Write Disk: {df['writ_dsk_total'].min()}")
print(f"Max Used Memory: {df['used_memory'].max()} ; Min Used Memory: {df['used_memory'].min()}")
print(f"Max Free Memory: {df['free_memory'].max()} ; Min Free Memory: {df['free_memory'].min()}")
print(f"Max Buffer Memory: {df['buff_memory'].max()} ; Min Buffer Memory: {df['buff_memory'].min()}")
print(f"Max Cache Memory: {df['cach_memory'].max()} ; Min Cache Memory: {df['cach_memory'].min()}")
print(f"Max Receive Net: {df['recv_net_total'].max()} ; Min Receive Net: {df['recv_net_total'].min()}")
print(f"Max Send Net: {df['send_net_total'].max()} ; Min Send Net: {df['send_net_total'].min()}")
print(f"Max User CPU Usage: {df['usr_cpu_usage'].max()} ; Min User CPU Usage: {df['usr_cpu_usage'].min()}")
print(f"Max System CPU Usage: {df['sys_cpu_usage'].max()} ; Min System CPU Usage: {df['sys_cpu_usage'].min()}")
print(f"Max Idle CPU Usage: {df['idl_cpu_usage'].max()} ; Min Idle CPU Usage: {df['idl_cpu_usage'].min()}")
print(f"Max Wait CPU Usage: {df['wai_cpu_usage'].max()} ; Min Wait CPU Usage: {df['wai_cpu_usage'].min()}")
print()
print (f"Max GPU Temperature: {df_gpu['temperature_gpu'].max()} ; Min GPU Temperature: {df_gpu['temperature_gpu'].min()}")
print (f"Max GPU Utilization (%): {df_gpu['utilization_gpu'].max()} ; Min GPU Utilization: {df_gpu['utilization_gpu'].min()}")
print (f"Max GPU Utilization Memory (MiB): {df_gpu['utilization_memory'].max()} ; Min GPU Utilization Memory (MiB): {df_gpu['utilization_memory'].min()}")
print (f"Max GPU Memory Total (MiB): {df_gpu['memory_total'].max()} ; Min GPU Memory Total (MiB): {df_gpu['memory_total'].min()}")
print (f"Max GPU Memory Free (MiB): {df_gpu['memory_free'].max()} ; Min GPU Memory Free (MiB): {df_gpu['memory_free'].min()}")
print (f"Max GPU Memory Used (MiB): {df_gpu['memory_used'].max()} ; Min GPU Memory Used (MiB): {df_gpu['memory_used'].min()}")

In [ ]:
# plots IO with time
plt.figure()
plt.plot(df['system_time'], df['read_io_total'], label='Reads')
plt.plot(df['system_time'], df['write_io_total'], label='Write Requests')
plt.xlabel('Time (s)')
plt.ylabel('#IO operations')
plt.title('IO with time')
plt.legend()
plt.show()

In [ ]:
# plots Disk with time
plt.figure()
plt.plot(df['system_time'], df['read_dsk_total'], label='Reads')
plt.plot(df['system_time'], df['writ_dsk_total'], label='Writes')
plt.xlabel('Time (s)')
plt.ylabel('Disk (bytes)')
plt.title('Disk with time')
plt.legend()
plt.show()

In [ ]:
# plots Memory with time
plt.figure()
plt.plot(df['system_time'], df['used_memory'], label='Used Memory')
plt.plot(df['system_time'], df['free_memory'], label='Free Memory')
plt.plot(df['system_time'], df['buff_memory'], label='Buffer Memory')
plt.plot(df['system_time'], df['cach_memory'], label='Cache Memory')
plt.xlabel('Time (s)')
plt.ylabel('Memory (bytes)')
plt.title('Memory with time')
plt.legend()
plt.show()

In [ ]:
# plots Network with time
plt.figure()
plt.plot(df['system_time'], df['recv_net_total'], label='Received Net')
plt.plot(df['system_time'], df['send_net_total'], label='Send Net')
plt.xlabel('Time (s)')
plt.ylabel('Network (bytes)')
plt.title('Network with time')
plt.legend()
plt.show()

In [ ]:
# plots CPU with time
plt.figure()
plt.plot(df['system_time'], df['usr_cpu_usage'], label='usr CPU usage')
plt.plot(df['system_time'], df['sys_cpu_usage'], label='sys CPU usage')
plt.plot(df['system_time'], df['idl_cpu_usage'], label='idl CPU usage')
plt.plot(df['system_time'], df['wai_cpu_usage'], label='wai CPU usage')
plt.xlabel('Time (s)')
plt.ylabel('CPU (%)')
plt.title('CPU with time')
plt.legend()
plt.show()

In [ ]:
# plots gpu temperature with time
plt.figure()
plt.plot(df_gpu['timestamp'], df_gpu['temperature_gpu'], label='Temperature')
plt.xlabel('Time (s)')
plt.ylabel('Temperature (°C)')
plt.title('GPU Temperature with time')
plt.legend()
plt.show()

In [ ]:
# plots gpu utilization with time
plt.figure()
plt.plot(df_gpu['timestamp'], df_gpu['utilization_gpu'], label='Utilization')
plt.xlabel('Time (s)')
plt.ylabel('Utilization (%)')
plt.title('GPU Utilization with time')
plt.legend()
plt.show()

In [ ]:
# plots gpu memory with time
plt.figure()
plt.plot(df_gpu['timestamp'], df_gpu['memory_total'], label='Total Memory')
plt.plot(df_gpu['timestamp'], df_gpu['memory_free'], label='Free Memory')
plt.plot(df_gpu['timestamp'], df_gpu['memory_used'], label='Used Memory')
plt.xlabel('Time (s)')
plt.ylabel('Memory (MiB)')
plt.title('GPU Memory with time')
plt.legend()
plt.show()